In [1]:
sam_f = '/home/ytanigaw/data/nanopore/20161008_wgs_caucasian_48hr.10k.bwa.mapq60.20kb.chr11.1.sam'

In [2]:
def read_sam(sam_f):
    with open(sam_f, 'r') as f:
        head = f.readline().strip()
        entry = f.readline().strip().split()
    return(head, entry)

In [3]:
def cigar2count(cigar_str):
    stats = {'M':0, 'I':0, 'D':0, 'N':0, 'S':0, 'H':0, 'P':0, '=':0, 'X':0}
    start = 0
    i = 0
    relative_pos = 0
    while(i < len(cigar_str)):
        while('0' <= cigar_str[i] <= '9'):
            i = i + 1
        segment_length = int(cigar_str[start:i])
        segment_type = cigar_str[i]
        relative_pos = relative_pos + segment_length
        stats[segment_type] = stats[segment_type] + segment_length
        i = i + 1
        start = i
    return(stats)

In [6]:
def get_count(sam_f):
    (head, entry) = read_sam(sam_f=sam_f)
    cnt = cigar2count(cigar_str=entry[5])
    return(head, entry, cnt)

In [7]:
(head, entry, cnt) = get_count(sam_f=sam_f)

In [8]:
cnt

{'=': 0,
 'D': 1940,
 'H': 0,
 'I': 247,
 'M': 14563,
 'N': 0,
 'P': 0,
 'S': 15350,
 'X': 0}

## scratch code follows

In [5]:
entry[0]

'channel_75_80bdccec-0a01-48bb-854d-094281ccc083_qscore_8.2_read_score_-1.8'

In [6]:
seq = entry[9]

In [7]:
pos = int(entry[3]) - 1

In [8]:
pos

9162868

In [30]:
cigar = entry[5]

In [33]:
cigar2count(cigar)

{'=': 0,
 'D': 1940,
 'H': 0,
 'I': 247,
 'M': 14563,
 'N': 0,
 'P': 0,
 'S': 15350,
 'X': 0}

In [25]:
stats = {'M':0, 'I':0, 'D':0, 'N':0, 'S':0, 'H':0, 'P':0, '=':0, 'X':0}
start = 0
i = 0
relative_pos = 0
while(i < len(cigar_t)):
    while('0' <= cigar_t[i] <= '9'):
        i = i + 1
    segment_length = int(cigar_t[start:i])
    segment_type = cigar_t[i]
    relative_pos = relative_pos + segment_length
    stats[segment_type] = stats[segment_type] + segment_length
    #print(relative_pos, segment_length, segment_type)    
    i = i + 1
    start = i
      

(21, 21, 'M')
(22, 1, 'I')
(33, 11, 'M')
(34, 1, 'I')
(35, 1, 'M')
(36, 1, 'I')
(39, 3, 'M')
(42, 3, 'I')
(50, 8, 'M')
(51, 1, 'D')
(71, 20, 'M')
(72, 1, 'D')
(83, 11, 'M')
(85, 2, 'D')
(98, 13, 'M')
(99, 1, 'I')
(111, 12, 'M')
(113, 2, 'D')
(129, 16, 'M')
(133, 4, 'D')
(140, 7, 'M')
(141, 1, 'D')
(145, 4, 'M')
(148, 3, 'I')
(158, 10, 'M')
(159, 1, 'I')
(160, 1, 'M')
(161, 1, 'I')
(180, 19, 'M')
(181, 1, 'I')
(193, 12, 'M')
(194, 1, 'D')
(228, 34, 'M')
(230, 2, 'D')
(242, 12, 'M')
(243, 1, 'D')
(244, 1, 'M')
(246, 2, 'D')
(259, 13, 'M')
(261, 2, 'D')
(267, 6, 'M')
(268, 1, 'D')
(277, 9, 'M')


In [26]:
stats

{'=': 0, 'D': 20, 'H': 0, 'I': 13, 'M': 244, 'N': 0, 'P': 0, 'S': 0, 'X': 0}

In [43]:
for type in ['N', 'S', 'H', 'P', '=', 'X']:
    print(type, type in cigar)

('N', False)
('S', True)
('H', False)
('P', False)
('=', False)
('X', False)


In [40]:
cigar_t.split(['M', 'I', 'D'])

TypeError: expected a character buffer object

## reference sequence
- we can read reference sequence if needed

In [15]:
import pysam

In [17]:
ref_f = '/share/PI/mrivas/data/hg19/hg19.fa'

In [18]:
hg19 = pysam.FastaFile(ref_f)

In [24]:
ref = hg19.fetch(reference = 'chr11', start = pos, end = pos + len(seq))

In [26]:
len(ref)

30160